# Request Generation Test Notebook

This notebook is created so a simple example of the request generation can be performed.

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go

### 1. Request Data Generation

The request generation is modelled in Request_generation.py. A object class is developed for each request type, passenger and parcel, since the stochasticity was implemented in a different way for each of them. The passenger requests are generated through a Non-homogeneous Poisson Process, while the parcel requests are probabilistically generated in terms of the occurring time. Please check out the respective source codes.
In order to change the parameters for the request generation, please check out the Data folder and modify the excel files accordingly.

In [2]:
# import the package
import Request_generation as rg

In [3]:
# Define the parameters

N_inst = 2    # Number of instances
Folder = '../..//Data/'    # Folder where the data is stored
Folder_save = '../Results/'    # Folder where the results will be saved

In [4]:
# Generate the requests
rg.main(N_inst, Folder, Folder_save)

Instance 0 and demand level low
Pattern [2, 2] is created and saved to the excel file
Pattern [1, 2] is created and saved to the excel file
Pattern [2, 1] is created and saved to the excel file
Pattern [0, 2] is created and saved to the excel file
Pattern [2, 0] is created and saved to the excel file
Pattern [0, 1] is created and saved to the excel file
Pattern [1, 0] is created and saved to the excel file
Instance 1 and demand level low
Pattern [2, 2] is created and saved to the excel file
Pattern [1, 2] is created and saved to the excel file
Pattern [2, 1] is created and saved to the excel file
Pattern [0, 2] is created and saved to the excel file
Pattern [2, 0] is created and saved to the excel file
Pattern [0, 1] is created and saved to the excel file
Pattern [1, 0] is created and saved to the excel file
Instance 0 and demand level high
Pattern [2, 2] is created and saved to the excel file
Pattern [1, 2] is created and saved to the excel file
Pattern [2, 1] is created and saved to 

### 2. Visualisation of request data

This part visualises the temporal distribution of the generated request data.

In [5]:
# Read the file names in the folder
filenames = os.listdir(Folder_save)
dist_files = [f for f in filenames if 'distribution' in f]    # list of the distribution files

In [6]:
print(dist_files)

['Request_distribution_instance_0_demand_high.xlsx', 'Request_distribution_instance_0_demand_low.xlsx', 'Request_distribution_instance_1_demand_high.xlsx', 'Request_distribution_instance_1_demand_low.xlsx']


In [7]:
# Example function to load and create a figure from a file
def load_data_and_create_figure(Folder, filename):
    # Load data
    df = pd.read_excel(Folder + filename)

    # Create the figure (cumulative line plots for requests)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Time_h'], y=df['Total_requests'].cumsum(), mode='lines', name="Total requests"))
    fig.add_trace(go.Scatter(x=df['Time_h'], y=df['Passenger_requests'].cumsum(), mode='lines', name="Passenger requests"))
    fig.add_trace(go.Scatter(x=df['Time_h'], y=df['Parcel_requests'].cumsum(), mode='lines', name="Parcel requests"))

    # Show time every 30 minutes
    fig.update_xaxes(tickvals=df['Time_h'][::30], ticktext=df['Time_h'][::30])
    
    return fig, df['Time_h']


# Create an empty figure
fig = go.Figure()

# Define the buttons for the dropdown
dropdown_buttons = []

# Add a figure trace for each file, making them initially invisible except the first file
for idx, file in enumerate(dist_files):
    # Create traces for each file using the function
    current_fig, time_h = load_data_and_create_figure(Folder_save, file)

    # Add all traces (3 traces per file) to the main figure, initially set visible to False
    for trace in current_fig.data:
        fig.add_trace(trace)
        # Set visibility of the first file's traces to True, and others to False
        fig.data[-1].visible = True if idx == 0 else False

# Add dropdown buttons for each file
for idx, file in enumerate(dist_files):
    # Make all three traces for the selected file visible
    visibility = [False] * len(fig.data)
    visibility[idx*3:(idx+1)*3] = [True, True, True]  # Make three traces visible for the selected file
    
    dropdown_buttons.append(
        dict(
            args=[{'visible': visibility}],
            label=file[:-5],  # Remove the extension from the filename
            method='update'
        )
    )

# Add dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_buttons,
            x=0,
            xanchor='center',
            y=1.3,
            yanchor='top',
        )
    ]
)

fig.update_layout(
    title='Cumulative distribution of requests',
    title_x=0.5,
    xaxis_title='Time',
    yaxis_title='Number of requests',
    xaxis_tickangle=60
)

# x axis is every 30 minutes
fig.update_layout(xaxis = dict(tickvals=time_h[::30], ticktext=time_h[::30]))
# Display the figure
fig.show()


The higher the arrival rate is during certain time period, the steeper the increment of the number of requests should be. In the study, the arrival rate has been set higher during the peak hours for passengers.  